In [25]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk_data_path = r'C:\Users\thori\AppData\Roaming\nltk_data'
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

# Download necessary NLTK resources
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('punkt_tab', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('omw-1.4', download_dir=nltk_data_path)
nltk.download('averaged_perceptron_tagger', download_dir=nltk_data_path)

# Load the data
file_path = r'complaints.xlsx.csv'  
df = pd.read_csv(file_path)

complaint_column = 'Consumer complaint narrative'  

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# Text cleaning function
def clean_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        
        # Remove redacted values (XXXX)
        text = re.sub(r'x+', '', text)
        
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords
        
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalpha()]
        
        # Join tokens back into a string
        return ' '.join(tokens)
    else:
        return ''

# Apply cleaning to the narrative column
df['cleaned_text'] = df[complaint_column].apply(clean_text)

# Save cleaned data to a new Excel file
df.to_excel('cleaned_complaints.xlsx', index=False)

# Create document-term matrices
# Bag-of-Words (CountVectorizer)
count_vectorizer = CountVectorizer(min_df=5, max_df=0.9)
count_matrix = count_vectorizer.fit_transform(df['cleaned_text'])
count_feature_names = count_vectorizer.get_feature_names_out()

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Save vectorized matrices to CSV
pd.DataFrame(count_matrix.toarray(), columns=count_feature_names).to_csv('bow_matrix.csv')
pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names).to_csv('tfidf_matrix.csv')

# Topic Modeling
# Number of topics - adjust based on your needs
n_topics = 5

# LDA with CountVectorizer
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_output = lda_model.fit_transform(count_matrix)

# LDA with TF-IDF
lda_tfidf_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_tfidf_output = lda_tfidf_model.fit_transform(tfidf_matrix)

# NMF with CountVectorizer
nmf_bow_model = NMF(n_components=n_topics, random_state=42)
nmf_bow_output = nmf_bow_model.fit_transform(count_matrix)

# NMF with TF-IDF
nmf_model = NMF(n_components=n_topics, random_state=42)
nmf_output = nmf_model.fit_transform(tfidf_matrix)

# Function to display top words for each topic
def display_topics(model, feature_names, n_top_words=10):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[:-n_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topics.append({f"Topic {topic_idx+1}": top_words})
    return topics

# Get top words for each topic
lda_topics = display_topics(lda_model, count_feature_names)
nmf_topics = display_topics(nmf_model, tfidf_feature_names)
lda_tfidf_topics = display_topics(lda_tfidf_model, tfidf_feature_names)
nmf_bow_topics = display_topics(nmf_bow_model, count_feature_names)

# Create DataFrames for topics
lda_topics_df = pd.DataFrame()
nmf_topics_df = pd.DataFrame()
lda_tfidf_df = pd.DataFrame()
nmf_bow_df = pd.DataFrame()
for topic_dict in lda_topics:
    for topic_name, words in topic_dict.items():
        lda_topics_df[topic_name] = pd.Series(words)

for topic_dict in nmf_topics:
    for topic_name, words in topic_dict.items():
        nmf_topics_df[topic_name] = pd.Series(words)

for topic_dict in lda_tfidf_topics:
    for topic_name, words in topic_dict.items():
        lda_tfidf_df[topic_name] = pd.Series(words)

for topic_dict in nmf_bow_topics:
    for topic_name, words in topic_dict.items():
        nmf_bow_df[topic_name] = pd.Series(words)


# Save topics to Excel
lda_topics_df.to_excel('lda_topics.xlsx', index=False)
nmf_topics_df.to_excel('nmf_topics.xlsx', index=False)
lda_tfidf_df.to_excel('lda_tfidf_topics.xlsx', index=False)
nmf_bow_df.to_excel('nmf_bow_topics.xlsx', index=False)

# Document-Topic Distribution
lda_document_topics = pd.DataFrame(lda_output, columns=[f'LDA_Topic_{i+1}' for i in range(n_topics)])
nmf_document_topics = pd.DataFrame(nmf_output, columns=[f'NMF_Topic_{i+1}' for i in range(n_topics)])
lda_tfidf_doc = pd.DataFrame(lda_tfidf_output, columns=[f'LDA_TFIDF_Topic_{i+1}' for i in range(n_topics)])
nmf_bow_doc = pd.DataFrame(nmf_bow_output, columns=[f'NMF_BoW_Topic_{i+1}' for i in range(n_topics)])

# Add document-topic distributions to the original dataframe
result_df = pd.concat([df, 
                       lda_document_topics, lda_tfidf_doc,
                       nmf_document_topics, nmf_bow_doc], axis=1)
result_df.to_excel('complaints_all_models.xlsx', index=False)

# Print summary
print(f"Total complaints processed: {len(df)}")
print("\nLDA Topics (Bag-of-Words):")
for topic in lda_topics:
    for topic_name, words in topic.items():
        print(f"{topic_name}: {', '.join(words)}")
print("\nLDA Topics (TF-IDF):")
for topic in lda_tfidf_topics:
    for topic_name, words in topic.items():
        print(f"{topic_name}: {', '.join(words)}")
print("\nNMF Topics (TF-IDF):")
for topic in nmf_topics:
    for topic_name, words in topic.items():
        print(f"{topic_name}: {', '.join(words)}")
print("\nNMF Topics (BoW):")
for topic in nmf_bow_topics:
    for topic_name, words in topic.items():
        print(f"{topic_name}: {', '.join(words)}")
print("\nAll results have been saved to Excel files.")

# Compute dominant topic per document for each model 
def get_dominant_topic(doc_topic_matrix):
    return doc_topic_matrix.idxmax(axis=1)  # Column with highest value

# Function to get dominant topic for each document
def get_dominant_topic(doc_topic_matrix):
    return doc_topic_matrix.idxmax(axis=1)  # Column with highest probability

# Compute dominant topics
result_df['LDA_BoW_Dominant'] = get_dominant_topic(lda_document_topics)
result_df['LDA_TFIDF_Dominant'] = get_dominant_topic(lda_tfidf_doc)
result_df['NMF_TFIDF_Dominant'] = get_dominant_topic(nmf_document_topics)
result_df['NMF_BoW_Dominant'] = get_dominant_topic(nmf_bow_doc)

# Topic label mappings
lda_bow_labels = {
    'LDA_Topic_1': 'Credit & Reporting',
    'LDA_Topic_2': 'Mortgage Servicing',
    'LDA_Topic_3': 'Payments & Accounts',
    'LDA_Topic_4': 'Insurance & Escrow',
    'LDA_Topic_5': 'Servicing Errors'
}

lda_tfidf_labels = {
    'LDA_TFIDF_Topic_1': 'Communication & Calls',
    'LDA_TFIDF_Topic_2': 'Mortgage Foreclosure',
    'LDA_TFIDF_Topic_3': 'Insurance & Escrow',
    'LDA_TFIDF_Topic_4': 'Payment Issues',
    'LDA_TFIDF_Topic_5': 'Credit Reporting'
}

nmf_tfidf_labels = {
    'NMF_Topic_1': 'Communication & Loan Info',
    'NMF_Topic_2': 'Mortgage Foreclosure',
    'NMF_Topic_3': 'Insurance & Property',
    'NMF_Topic_4': 'Payments & Fees',
    'NMF_Topic_5': 'Credit Reporting'
}

nmf_bow_labels = {
    'NMF_BoW_Topic_1': 'Credit Reporting',
    'NMF_BoW_Topic_2': 'Mortgage & Servicing',
    'NMF_BoW_Topic_3': 'Payment Issues',
    'NMF_BoW_Topic_4': 'Insurance & Escrow',
    'NMF_BoW_Topic_5': 'Servicing Errors'
}

# Map dominant topic number to label
result_df['LDA_BoW_Label'] = result_df['LDA_BoW_Dominant'].map(lda_bow_labels)
result_df['LDA_TFIDF_Label'] = result_df['LDA_TFIDF_Dominant'].map(lda_tfidf_labels)
result_df['NMF_TFIDF_Label'] = result_df['NMF_TFIDF_Dominant'].map(nmf_tfidf_labels)
result_df['NMF_BoW_Label'] = result_df['NMF_BoW_Dominant'].map(nmf_bow_labels)

# Print topic prevalence for all models in desired format
print("\nTopic prevalence comparison (all models):\n")

model_label_pairs = [
    ('LDA_BoW_Label', lda_bow_labels),
    ('LDA_TFIDF_Label', lda_tfidf_labels),
    ('NMF_TFIDF_Label', nmf_tfidf_labels),
    ('NMF_BoW_Label', nmf_bow_labels)
]

for col, label_map in model_label_pairs:
    counts = result_df[col].value_counts()
    print(f"{col} Topic Prevalence:")
    # Print with Topic X: Label: N complaints
    for i, (label, count) in enumerate(counts.items(), start=1):
        # Find topic number from label_map (reverse lookup)
        topic_num = [k.split('_')[-1] for k,v in label_map.items() if v == label][0]
        print(f"Topic {topic_num}: {label}: {count} complaints")
    print("")  # Blank line for separation

# Save results with labels to Excel
result_df.to_excel('complaints_all_models_with_labels.xlsx', index=False)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\thori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thori\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\thori\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thori\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Total complaints processed: 903

LDA Topics (Bag-of-Words):
Topic 1: credit, forbearance, reporting, reported, information, disaster, loss, loan, mitigation, bureau
Topic 2: shellpoint, mortgage, servicing, loan, payment, foreclosure, complaint, year, error, account
Topic 3: payment, mortgage, loan, account, year, shellpoint, would, time, told, month
Topic 4: insurance, escrow, newrez, mortgage, property, policy, document, loan, shellpoint, company
Topic 5: servicing, payment, newrez, cfpb, failure, violation, usc, borrower, unapplied, ledger

LDA Topics (TF-IDF):
Topic 1: overage, referring, word, pertaining, delivered, joint, belong, miscommunication, maintenance, recalculated
Topic 2: overage, referring, word, eadacpa, pertaining, delivered, joint, belong, miscommunication, maintenance
Topic 3: payment, shellpoint, mortgage, loan, escrow, year, account, newrez, insurance, company
Topic 4: overage, referring, word, pertaining, delivered, joint, belong, miscommunication, maintenance, 